# Clase 2: Seguimiento y comparación de modelos con MLflow

Dataset: Churn de clientes - Telco

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

url = "..\data\WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df.drop(['customerID'], axis=1, inplace=True)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)
df = pd.get_dummies(df, drop_first=True)

X = df.drop('Churn_Yes', axis=1)
y = df['Churn_Yes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from scipy.stats import uniform
from sklearn.svm import SVC

mlflow.set_experiment("Churn_Telco_Comparacion")

<Experiment: artifact_location='file:///c:/MLOPS/Proyecto_1/notebooks/mlruns/310794751774304322', creation_time=1744329200676, experiment_id='310794751774304322', last_update_time=1744329200676, lifecycle_stage='active', name='Churn_Telco_Comparacion', tags={}>

### Entrenamiento: Logistic Regression

In [5]:
with mlflow.start_run(run_name="Logistic Regression"):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    mlflow.log_param("modelo", "logistic")
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))
    
    mlflow.sklearn.log_model(model, "modelo_logistico")

c:\MLOPS\env\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/10 18:53:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Entrenamiento: Random Forest

In [6]:
with mlflow.start_run(run_name="Random Forest"):
    model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    mlflow.log_param("modelo", "random_forest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 6)
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))
    
    mlflow.sklearn.log_model(model, "modelo_rf")

2025/04/10 18:53:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Entrenamiento: Support vector machines

In [ ]:
param_dist = {
    'C': uniform(0.1, 10),       # Random float between 0.1 and 10
    'kernel': ['linear', 'rbf'], # Random choice
    'gamma': ['scale', 'auto']   # Random choice
}

# Randomized Search
random_search = RandomizedSearchCV(
    SVC(probability=True, random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # Number of random combinations
    cv=5,
    scoring='accuracy',
    random_state=42
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

In [ ]:
with mlflow.start_run(run_name="Support Vector Machine"):
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    # Log all best hyperparameters
    mlflow.log_param("modelo", "support_vector_machine")
    for param_name, param_value in best_params.items():
        mlflow.log_param(param_name, param_value)

    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))

    mlflow.sklearn.log_model(best_model, "modelo_svc")

2025/04/10 19:29:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Visualizar Experimentos en la UI de MLflow

In [7]:
# Ejecuta en terminal (no dentro del notebook):
# mlflow ui
# Luego abre en el navegador: http://127.0.0.1:5000